In [1]:
from TCs_class_methods import TCs, Testwarn
from aqua.util import load_yaml

Load configuration file with all the tempest options

In [2]:
tdict = load_yaml('config/config_levante.yml')
tdict

{'dataset': {'model': 'IFS', 'exp': 'tco2559-ng5'},
 'paths': {'regdir': '/scratch/b/b382216/regrid_new',
  'tmpdir': '/scratch/b/b382216/tmpdir_new',
  'fulldir': '/scratch/b/b382216/fullres_new'},
 'grids': {'lowgrid': 'r100', 'highgrid': 'r010'},
 'varlist': ['msl', '10u', '10v', 'tp'],
 'time': {'start_date': '2020-08-01',
  'end_date': '2020-09-30',
  'frequency': '6h'},
 'detect': {'boxdim': 10},
 'stitch': {'n_days_freq': 25, 'n_days_ext': 15}}

Initialise the class with the dictionary from the YAML file

In [3]:
warnings = Testwarn(loglevel='ERROR')
tropical = TCs(tdict=tdict, streaming=True, stream_step=3, stream_unit="days", stream_startdate=tdict['time']['start_date'], loglevel = "INFO")
#tropical = TCs(paths=tdict['paths'], var2store=tdict['varlist'])

In [7]:
tropical.catalog_init(streaming=tropical.streaming)

2023-04-12 10:10:40 :: TCs :: WARNING  -> Initialised streaming for 3 days starting on 2020-08-01


Run the code on a loop on the detect nodes

In [4]:
tropical.detect_nodes_zoomin(start_date=tdict['time']['start_date'], 
                             end_date=tdict['time']['end_date'], 
                             frequency=tdict['time']['frequency'])

2023-04-12 10:14:30 :: TCs :: WARNING  -> Initialised streaming for 3 days starting on 2020-08-01
2023-04-12 10:14:58 :: TCs :: WARNING  -> 20200801T00
2023-04-12 10:14:58 :: TCs :: INFO     -> Running readwrite_from_intake() for 20200801T00
2023-04-12 10:15:03 :: TCs :: INFO     -> Accessing 2D data..
2023-04-12 10:15:03 :: TCs :: INFO     -> Accessing 2D data..
2023-04-12 10:15:03 :: TCs :: INFO     -> Accessing 2D data..


<xarray.DataArray 'time' (time: 72)>
array(['2020-08-01T00:00:00.000000000', '2020-08-01T01:00:00.000000000',
       '2020-08-01T02:00:00.000000000', '2020-08-01T03:00:00.000000000',
       '2020-08-01T04:00:00.000000000', '2020-08-01T05:00:00.000000000',
       '2020-08-01T06:00:00.000000000', '2020-08-01T07:00:00.000000000',
       '2020-08-01T08:00:00.000000000', '2020-08-01T09:00:00.000000000',
       '2020-08-01T10:00:00.000000000', '2020-08-01T11:00:00.000000000',
       '2020-08-01T12:00:00.000000000', '2020-08-01T13:00:00.000000000',
       '2020-08-01T14:00:00.000000000', '2020-08-01T15:00:00.000000000',
       '2020-08-01T16:00:00.000000000', '2020-08-01T17:00:00.000000000',
       '2020-08-01T18:00:00.000000000', '2020-08-01T19:00:00.000000000',
       '2020-08-01T20:00:00.000000000', '2020-08-01T21:00:00.000000000',
       '2020-08-01T22:00:00.000000000', '2020-08-01T23:00:00.000000000',
       '2020-08-02T00:00:00.000000000', '2020-08-02T01:00:00.000000000',
       '2020-0

2023-04-12 10:15:10 :: TCs :: INFO     -> Accessing 3D data..
2023-04-12 10:15:10 :: TCs :: INFO     -> Writing low res to disk..
2023-04-12 10:15:13 :: TCs :: INFO     -> Running run_detect_nodes() for 20200801T00
2023-04-12 10:15:13 :: TCs :: INFO     -> Running store_detect_nodes() for 20200801T00
2023-04-12 10:15:36 :: TCs :: WARNING  -> 20200801T06
2023-04-12 10:15:36 :: TCs :: INFO     -> Running readwrite_from_intake() for 20200801T06
2023-04-12 10:15:37 :: TCs :: INFO     -> Accessing 2D data..


<xarray.DataArray 'time' (time: 72)>
array(['2020-08-04T00:00:00.000000000', '2020-08-04T01:00:00.000000000',
       '2020-08-04T02:00:00.000000000', '2020-08-04T03:00:00.000000000',
       '2020-08-04T04:00:00.000000000', '2020-08-04T05:00:00.000000000',
       '2020-08-04T06:00:00.000000000', '2020-08-04T07:00:00.000000000',
       '2020-08-04T08:00:00.000000000', '2020-08-04T09:00:00.000000000',
       '2020-08-04T10:00:00.000000000', '2020-08-04T11:00:00.000000000',
       '2020-08-04T12:00:00.000000000', '2020-08-04T13:00:00.000000000',
       '2020-08-04T14:00:00.000000000', '2020-08-04T15:00:00.000000000',
       '2020-08-04T16:00:00.000000000', '2020-08-04T17:00:00.000000000',
       '2020-08-04T18:00:00.000000000', '2020-08-04T19:00:00.000000000',
       '2020-08-04T20:00:00.000000000', '2020-08-04T21:00:00.000000000',
       '2020-08-04T22:00:00.000000000', '2020-08-04T23:00:00.000000000',
       '2020-08-05T00:00:00.000000000', '2020-08-05T01:00:00.000000000',
       '2020-0

Traceback (most recent call last):
  File "pandas/_libs/index.pyx", line 548, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 2263, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 2273, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 1596261600000000000

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/work/bb1153/b382216/mambaforge/envs/TCs/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 516, in pandas._libs.index.DatetimeEngine.get_loc
  File "pandas/_libs/index.pyx", line 550, in pandas._libs.index.DatetimeEngine.get_loc
KeyError: Timestamp('2020-08-01 06:00:00')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/work/bb1153/

Run the code for the detect nodes, after having set up the days freq and the extensions days

In [4]:
tropical.stitch_nodes_zoomin(start_date=tdict['time']['start_date'], end_date=tdict['time']['end_date'],
                             n_days_freq=tdict['stitch']['n_days_freq'], n_days_ext=tdict['stitch']['n_days_ext'])

2023-03-30 19:12:52 WARNING: 2020-08-01 00:00:00


msl
2020080306
2020080312
2020080318
2020080400
2020080406
2020080412
2020080418
2020080500
2020080506
2020080512
2020080518
2020080600
2020080606
2020080612
2020080618
2020080700
2020080706
2020080712
2020080718
2020080800
2020080806
2020080812
2020080818
2020080900
2020080906
2020080912
2020080918
2020081000
2020081006
2020081012
2020081018
2020081100
2020081106
2020081112
2020081118
2020081200
2020081206
2020081212
2020081218
2020081300
2020081306
2020081312
2020081318
2020081400
2020081406
2020081412
2020082400
2020082406
2020082412
2020082418
2020082500
2020082506
2020082512
2020082518
2020082600
2020082606
2020082612
2020082618
2020082700
2020082706
2020082712
2020082718
2020082800
2020082806
2020082812
2020082818
2020082900
2020082906
2020082912
2020082918
2020083000
2020083006
2020083012
2020083018
2020083100
2020083106
2020083112
2020083118
2020090100
2020090106
2020090112
2020090118
2020090200
2020090206
2020090212
2020090218
2020090300
2020090306
2020090312
2020090318
202009

2023-03-30 19:14:58 WARNING: 2020-08-26 00:00:00


msl
2020081100
2020081106
2020081112
2020081118
2020081200
2020081206
2020081212
2020081218
2020081300
2020081306
2020081312
2020081318
2020081400
2020081406
2020081412
2020082400
2020082406
2020082412
2020082418
2020082500
2020082506
2020082512
2020082518
2020082600
2020082606
2020082612
2020082618
2020082700
2020082706
2020082712
2020082718
2020082800
2020082806
2020082812
2020082818
2020082900
2020082906
2020082912
2020082918
2020083000
2020083006
2020083012
2020083018
2020083100
2020083106
2020083112
2020083118
2020090100
2020090106
2020090112
2020090118
2020090200
2020090206
2020090212
2020090218
2020090300
2020090306
2020090312
2020090318
2020090400
2020090406
2020090412
2020090418
2020090500
2020090506
2020090512
2020090518
2020090600
2020090606
2020090612
2020090618
2020090700
2020090706
2020090800
2020090806
2020090812
2020090818
2020090900
2020090906
2020090912
2020090918
2020091000
2020091006
2020091012
2020091018
2020091100
2020091106
2020091112
2020091118
2020091200
202009